<a id="top"></a>

# Processing WFC3/UVIS Data with `calwf3` Using the v1.0 CTE-Correction  
***
## Learning Goals
This notebook explains how to calibrate `raw` WFC3/UVIS data with the v1.0 pixel-based CTE correction within `calwf3`. 
<br>

By the end of this tutorial, you will:

- Download a raw WFC3 image from MAST.
- Find the necessary reference files needed for calibration.
- Edit header keywords.
- Run `calwf3` `v3.5.2` to calibrate the raw image with the v1.0 pixel based CTE-correction.
- Compare v1.0 and v2.0 products. 


Please make sure you have read the `README.md` file before continuing.

## Table of Contents
 [Introduction](#intro)<br>
 
 [1. Imports](#imports) <br>
 [2. Verify `archived_drkcfiles.txt` is in CWD](#txtfile)<br>
 [3. Check that `calwf3` Version is `v3.5.2`](#checkcalver)<br>
 [4. Query MAST and Download a WFC3 `raw.fits` Image](#download) <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.1 Inspect Image Headers](#imageheaders) <br>
 [5. Find the Correct `DRKCFILE`](#finddark) <br>
 [6  Modify Image Header](#modhead) <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[6.1 `DRKCFILE`](#drkcfile) <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[6.2 `PCTETAB`](#pctetab) <br>
[7. Re-Inspect Image Header](#reinspect) <br>
[8. Set Environment Variables](#envvar) <br>
[9. Run `calwf3`](#runcal) <br>
[10. Inspect `FLC` Image Header](#inspectflc) <br>
[11. Investigate v1.0 and v2.0 Differences](#verdiffs) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[11.1 Download the v2.0 FLC File](#downloadv2.0)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[11.2 Open Files](#openfiles) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[11.3 Display 50x50 Pixel Background Subsection](#bkgsub) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[11.3.1 Pixel Distribution of Background Subsections](#bkgdist) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[11.4 Display Image Subsection](#imgsub) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[11.5 Aperture Photometry](#apphot) <br>
[12. Conclusions](#conclude) <br>

[Additional Resources](#resources) <br>
[About the Notebook](#about) <br>
[Citations](#cite) 


<a id="intro"></a>

## Introduction 

The v1.0 pixel-based Charge Transfer Efficiency (CTE) correction was first implemented into `calwf3` `v3.3` in 2016 ([Ryan et al. 2016](https://ui.adsabs.harvard.edu/abs/2016wfc..rept....1R/abstract),<br> [Anderson & Bedin 2010](https://ui.adsabs.harvard.edu/abs/2010PASP..122.1035A/abstract), [HSTCAL release notes](https://github.com/spacetelescope/hstcal/releases/tag/1.0.0)). This also marked the first time users could directly download CTE-corrected `flc & drc` <br> files from [MAST](https://mast.stsci.edu/search/hst/ui/#/).  While the v1.0 correction was sufficient  for many years, the degradation of CTE over time reduced the efficacy of the model <br> in treating low-level pixels. The v1.0 correction adversely impacts (overcorrects) both the image background and faint sources. In April 2021 <br> the v2.0 pixel-based CTE correction was implemented in `calwf3` `v3.6.0` ([Anderson et al. 2021](https://ui.adsabs.harvard.edu/abs/2021wfc..rept....9A/abstract), [Kuhn & Anderson 2021](https://ui.adsabs.harvard.edu/abs/2021wfc..rept....6K/abstract),  [HSTCAL release <br>notes](https://github.com/spacetelescope/hstcal/releases/tag/2.7.0)). Since MAST uses the latest release of `calwf3` for calibration, any WFC3/UVIS CTE corrected data retrieved from MAST, regardless of <br> observation date, will be calibrated with the v2.0 pixel-based CTE correction. Although v1.0 pixel-based CTE-corrected `flc & drc` files are <br> no longer accessable through MAST, this notebook steps through the procedure required to calibrate WFC3/UVIS images using the v1.0 CTE <br> correction. <br>


One of the limiting factors of using the v1.0 CTE correction are the CTE corrected dark current reference files (`DRKCFILE`). These dark <br> reference files are delivered to MAST by the WFC3 team and use the same pixel-based CTE correction within `calwf3`. Now that we <br> have switched to the v2.0 CTE correction there is a cut off for dark current reference files that use the v1.0 correction. Observations taken <br> after February 2021 will not have
CTE corrected dark files using the v1.0 algorithm, which means **applying the v1.0 CTE correction works <br> best for observations taken between May 2009 - February 2021.** If the observation being calibrated was taken after February 2021 there <br> are two options: **1)** use the last v1.0 CTE corrected dark reference file from February 2021 or **2)** use the v2.0 CTE corrected dark with the <br> most appropriate `USEAFTER` for the science exposure's observation date. 

<a id="imports"></a>
## 1. Imports

This notebook assumes you have created the virtual environment in [WFC3 Library's](https://github.com/spacetelescope/WFC3Library) installation instructions.

We import: <br>
 <br>
**•** *glob* for creating list of files  <br>
**•** *matplotlib.pyplot* for plotting and displaying images <br>
**•** *numpy* for finding indices and concatenating arrays <br>
**•** *os* for setting environment variables <br>
**•** *shutil* removing an empty directory <br>

**•** *astropy.io.fits* for opening and modifying fits files <br>
**•** *astroquery.mast.Observations* for downloading data from MAST <br>
**•** *astropy.table.Table* for creating and manipulating data tables<br>
**•** *astropy.time.Time* for converting between time formats<br>
**•** *ginga.util.zscale* for finding scale limits when displaying images <br>
**•** *photutils.aperture.aperture_photometry* for performing aperture photometry<br>
**•** *photutils.aperture.CircularAperture* for creating circular apertures <br>
**•** *photutils.aperture.CircularAnnulus* for creating circular annuli <br>
**•** *wfc3tools.calwf3* for verifying the version and running pipeline <br>

**•** *background_median.aperture_stats_tbl* for measuring background values within annuli <br>

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

from astropy.io import fits
from astroquery.mast import Observations
from astropy.table import Table
from astropy.time import Time
from ginga.util import zscale
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus
from wfc3tools import calwf3

from example.background_median import aperture_stats_tbl

<a id="txtfile"></a>
## 2. Verify `archived_drkcfiles.txt` is in CWD
When you cloned/downloaded this notebook from [WFC3 Library](https://github.com/spacetelescope/WFC3Library), a .txt file should have been included. The file name is <br> `archived_drkcfiles.txt` and it is used later on in the notebook. This .txt file includes the file name, delivery date, activation <br> date, and USEAFTER date for every v1.0 CTE corrected dark reference file between May 2009 - February 2021. Below, we will use <br> this file in conjunction with the observation date of the file(s) being calibrated to pick out the most appropriate v1.0 CTE corrected <br> dark reference file(s).<br>

Please make sure the file is in the current working directory before continuing.

In [ ]:
# list cwd to verify txt file is there
!ls -l archived_drkcfiles.txt


<a id="checkcalver"></a>
## 3. Check that `calwf3` Version is `v3.5.2`
In April 2021, a new `calwf3` version was released that contains the v2.0 CTE-correction.
If you would like to use the v1.0 correction, your <br> current environment must be using `calwf3` versions equal to or between `3.3` - `3.5.2`. However, in order to get the best v1.0 calibrated <br>images we must use `calwf3` `v3.5.2`. This version of `calwf3` includes the recent ([~Jan 2021](https://github.com/spacetelescope/hstcal/releases/tag/2.5.0)) update that added MJD as a parameterized <br> variable for the `PHOTMODE` keyword, which enables a time-dependent photometric correction and zeropoint.  If your version is `3.6.0` or higher, <br> you must downgrade the [`hstcal` package](https://github.com/spacetelescope/hstcal). The safer option, however, is to create a new environment such as: <br>

* `$ conda config --add channels http://ssb.stsci.edu/astroconda`
* `$ conda create -n v1_PCTE hstcal==2.5.0 python=3.7 ginga stsci-hst notebook`

`hstcal` `v2.5.0` provides version `3.5.2` of `calwf3`, which is the last version that offers the v1.0 pixel-based CTE correction.


In [ ]:
# print calwf3 version to make sure its equal to or between 3.3 and 3.5.2 
!calwf3.e --version

<a id="download"></a>
## 4. Query MAST and Download a WFC3 `raw.fits` Image 
Here, we download our image via `astroquery`. For more information, please look at the documentation for [Astroquery](https://astroquery.readthedocs.io/en/latest/),
[Astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html), and <br> 
[CAOM Field Descriptions](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html), which is used for the `obs_table` variable below. Additionally, you may download the data from MAST using either <br> the [HST MAST Search Engine](https://mast.stsci.edu/search/hst/ui/#/) or the more general [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html).

We download a `raw` image of star cluster 47 Tucanae (47Tuc, NGC 104), offset from the core, from CAL proposal [15576](https://www.stsci.edu/cgi-bin/get-proposal-info?id=15576&submit=Go&observatory=HST) (July 2019).<br>
After downloading the image, we move it to the current working directory (cwd).

In [ ]:
# Edit this cell's first line if you would to download your own file(s)
# Get the observation records
obs_table = Observations.query_criteria(obs_id='idv404axq*',proposal_id=15576)

# Get the listing of data products
products = Observations.get_product_list(obs_table)

# Filter the products for the RAW files
filtered_products = Observations.filter_products(products, productSubGroupDescription='RAW')

# Download all the images above
download_table = Observations.download_products(filtered_products, mrp_only=False)

# For convenience move raws to cwd and remove empty download dir
for file in download_table['Local Path']:
    filename = file.split('/')[-1]
    os.rename(file, os.path.basename(file))
    os.rmdir('mastDownload/HST/'+filename[:9])
        
os.rmdir('mastDownload/HST/')
os.rmdir('mastDownload/')
    

In [ ]:
# show list of current dir to verify fits file is there
!ls -l *raw.fits


<a id="imageheaders"></a>
## 4.1 Inspect Image Header
When processing a raw file through `calwf3` ,  the pipeline uses a few different header keywords to initiate and run the pixel-based CTE <br> correction. Here, we inspect the important header keywords from the raw file just downloaded.<br>
At this step, you should see: <br>
- `pctetab` set to `iref$54l1347ei_cte.fits`<br>
- `drkcfile` set to `iref$54n2022fi_dkc.fits`<br>
- `pctecorr` set to `PERFORM`.


In [ ]:
# Collect header keyword info from raw file
file, date, expstart, pctetab, drkcfile, pctecorr = [],[],[],[],[],[]
for f in glob.glob('*raw.fits'):
    h = fits.getheader(f)
    file.append(h['filename'])
    date.append(h['date-obs'])
    expstart.append(h['expstart'])
    pctetab.append(h['pctetab'])
    drkcfile.append(h['drkcfile'])
    pctecorr.append(h['pctecorr'])

image_table = Table([file, date, expstart, pctetab, drkcfile, pctecorr],
                    names=('file','date-obs', 'expstart', 'pctetab', 'drkcfile', 'pctecorr'))
image_table['expstart'].format = '5.6f'

# Sort and display the table
image_table.sort('expstart')
image_table

<a id="finddark"></a>
## 5. Find the Correct `DRKCFILE` 
Below, we open the .txt file containing a list of all the `DRKCFILE` reference files created with the v1.0 pixel-based CTE correction. <br>`DRKCFILE` reference files are CTE corrected files used by the pipeline to perform the dark current subtraction during the generation <br> of the `flc` file. The `DRKCFILE` files listed in `archived_drkcfiles.txt` have been archived on the CRDS database and while <br> they are still accessible for use and download, they are not being actively used by MAST. <br>

In the first cell, we generate an `astropy.Table` ( `drkc_table` ) using the data from the file `archived_drkcfiles.txt`, mentioned <br> in [Section 2](#txtfile), and create empty lists for the final table. Then, in the second cell, we index the `drkc_table` table for the best `DRKCFILE` <br> that corresponds to the `DATE-OBS`  of the `raw` file being calibrated. Lastly, in the third cell, we create and display the final `astropy.table` <br>that contains just the necessary `DRKCFILE`.<br> 



In [ ]:
# Generate astropy table from `archived_drkcfiles.txt`
drkc_table = Table.read('archived_drkcfiles.txt',format='ascii.commented_header')

# Create empty lists for final astropy table
rawfiles, obsdates, dkcfiles, uafters, active_dates = [],[],[],[], []


In [ ]:
# Using image header table from section 4.1 find closest drkcfile
for expstart in image_table['expstart']:
    table_idx = np.where(abs(drkc_table['useafter-mjd']-expstart) == abs(drkc_table['useafter-mjd']-expstart).min())[0][0]

    rawfile = image_table[image_table['expstart']==expstart]['file'][0]
    
    # if drkcfile has useafter date > rawfile expstart use previous drkcfile
    if drkc_table[table_idx]['useafter-mjd'] > image_table[image_table['file']==rawfile]['expstart'][0]:
        table_idx -= 1
    #append info
    rawfiles.append(rawfile)
    obsdates.append(image_table[image_table['file']==rawfile]['date-obs'][0])
    dkcfiles.append(drkc_table[table_idx]['drkcfile'])
    uafters.append(drkc_table[table_idx]['useafter'])
    active_dates.append(drkc_table[table_idx]['activation-date'])
    

In [ ]:
# Generate table of filename, date-obs, drkc-filename, corresponding useafter
raw_dkc_tab = Table([rawfiles, obsdates, dkcfiles, uafters, active_dates],
                    names=('filename','date-obs','dkc-filename','dkc-useafter','dkc-activation'))
# Display table
raw_dkc_tab


<a id="modhead"></a>
## 6. Modify Image Header
Now that we know which v1.0 CTE corrected dark current reference file corresponds to our `raw` science file, we're ready to edit the header <br> keywords the v1.0 CTE corrected reference files and tables. 

<a id="drkcfile"></a>
## 6.1 `DRKCFILE`
First, we will edit `raw` file's header with the v1.0 CTE corrected dark current reference file, `DRKCFILE`, that we just found in Section 6. 


In [ ]:
 for file in glob.glob('i*raw.fits'):
    # Using raw_dkc_tab from above, grab appropriate drkcfile
    ctecorr_dark = 'iref$'+raw_dkc_tab[raw_dkc_tab['filename']==file]['dkc-filename'][0]
    fits.setval(file, 'DRKCFILE', value = ctecorr_dark)


<a id="pctetab"></a>
## 6.2 `PCTETAB`
The next keyword we're going to modify is the `PCTETAB`. This is the pixel-based correction reference table and without it the algorithm will <br> not work. In order to use the v1.0 pixel based correction, `PCTETAB` will need to be set to the proper reference  file. In the below cell, we set <br> the `raw` file's `PCTETAB` to the v1.0 reference table, [zcv2057mi_cte.fits](https://hst-crds.stsci.edu/browse/zcv2057mi_cte.fits).


In [ ]:
fits.setval('idv404axq_raw.fits', 'PCTETAB', value = 'iref$zcv2057mi_cte.fits') 


<a id="reinspect"></a>
## 7. Re-Inspect Image Header 
Now with the headers modified, we inspect the keywords one last time to verify the file was updated properly before we process <br> it through `calwf3`. At this point you should see:<br> 
- `PCTETAB` set to `iref$zcv2057mi_cte.fits	`<br>
- `DRKCFILE` set to `iref$3961719li_dkc.fits`<br>

In [ ]:
# Recollect and display header keywords
file, date, expstart, pctetab, drkcfile, pctecorr = [],[],[],[],[],[]
for f in glob.glob('*raw.fits'):
    h = fits.getheader(f)
    file.append(h['filename'])
    date.append(h['date-obs'])
    expstart.append(h['expstart'])
    pctetab.append(h['pctetab'])
    drkcfile.append(h['drkcfile'])
    pctecorr.append(h['pctecorr'])

updated_table = Table([file, date, expstart, pctetab, drkcfile, pctecorr],
                    names=('file','date-obs', 'expstart', 'pctetab', 'drkcfile', 'pctecorr'))
updated_table['expstart'].format = '5.6f'

# Sort and display the table
updated_table.sort('expstart')
updated_table

<a id="envvar"></a>
## 8. Set Environment Variables
<div class="alert alert-block alert-warning" style="color:black" > <b> If you already have the absolute paths set for CRDS, please skip this step and proceed to Section 9.</b> <br> </div>
    
Before we run `calwf3`, we need to [set environment variables](https://hst-crds.stsci.edu/docs/cmdline_bestrefs/) for several subsequent calibration tasks.  We will point to a subdirectory called <br> `crds_cache/` using the `IREF` environment variable. The `IREF` variable is used for WFC3 reference files. Other instruments use other <br> variables, e.g., `JREF` for ACS. You have the option to permanently add these environment variables to your user profile by adding the path <br> in your shell's configuration file. If you're using bash, you would edit the `~/.bash_profile` file with lines such as:

* `export CRDS_PATH="$HOME/crds_cache"`
* `export CRDS_SERVER_URL="https://hst-crds.stsci.edu"`
* `export iref="${CRDS_PATH}/references/hst/iref/"`




In [ ]:
os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = '~/crds_cache'
os.environ['iref'] = '~/crds_cache/references/hst/iref/'


<a id="runcal"></a>
## 9. Run `calwf3` 
As a reminder, the `calwf3` version must be `3.5.2` to use the v1.0 pixel-base CTE correction with the most up-to-date calibration parameters. <br> If you are not using version `3.5.2` the below `calwf3` call will crash due to an out-of-date `IMPHTTAB` reference file. <br>

In [ ]:
calwf3('idv404axq_raw.fits')


In [ ]:
# show list of cwd to verify calibrated files were made
!ls -ltr *.fits


<a id="inspectflc"></a>
## 10. Inspect FLC Image Header
To verify that the data was calibrated with the v1.0 pixel-based CTE-correction, header keyword `CAL_VER` should be `3.5.2`,<br> `CTE_VER` should be `1.0`, and `CTE_NAME` should be `pixelCTE 2012`

In [ ]:
#Recollect and display FLC header keywords
file, pctetab, drkcfile, pctecorr, calver, ctename, ctever = [],[],[],[],[],[],[]
for f in glob.glob('*flc.fits'):
    h = fits.getheader(f)
    file.append(h['filename'])
    pctetab.append(h['pctetab'])
    drkcfile.append(h['drkcfile'])
    pctecorr.append(h['pctecorr'])
    calver.append(h['cal_ver'])
    ctename.append(h['cte_name'])
    ctever.append(h['cte_ver'])

final_table = Table([file, pctetab, drkcfile, pctecorr, calver, ctename, ctever],
                    names=('file','pctetab', 'drkcfile', 'pctecorr','cal_ver','cte_name','cte_ver'))

final_table

<a id="verdiffs"></a>
## 11. Investigate v1.0 and v2.0 Differences 
Now that we have created a v1.0 CTE corrected image, lets compare it to the same image calibrated with the v2.0 correction.<br> 

<a id="downloadv2.0"></a>
## 11.1 Download the v2.0 FLC File
First, we need to download the same FLC file from MAST that is corrected with the v2.0 pixel-based CTE correction so that we can <br> compare it to the v1.0 FLC file we just created in the notebook.

In [ ]:
# Get the observation records
obs_table = Observations.query_criteria(obs_id='idv404axq*',proposal_id=15576)

# Get the listing of data products
products = Observations.get_product_list(obs_table)

# Filter the products for the RAW files
filtered_products = Observations.filter_products(products, productSubGroupDescription='FLC',project='CALWF3')

# Download all the images above
download_table = Observations.download_products(filtered_products, mrp_only=False)

# For convenience move raws to cwd and remove empty download dir
for file in download_table['Local Path']:
    filename = file.split('/')[-1][:9]+'_v2.0_flc.fits'
    os.rename(file, os.path.basename(filename))
    os.rmdir('mastDownload/HST/'+filename[:9])
        
os.rmdir('mastDownload/HST/')
os.rmdir('mastDownload/')

<a id="openfiles"></a>
## 11.2 Open Files 
In the cell below, we open each of the different files (FLT, v1.0 and v2.0 FLC) and create full-frame ~4Kx4K arrays. Additionally, <br> we multiply the science arrays by the [pixel area maps](https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/pixel-area-maps). Due to the geometric distortion  present in WFC3 images, these pixel <br> area maps are necessary to achieve uniformity in the measured counts of an object across the field. The pixel map simply <br> reflects the area of the pixels at the location of the source, and by multiplying the images by this field-dependent correction <br> factor, we will improve the accuracy of the photometry.    

In [ ]:
# Open data and set variables
with fits.open('idv404axq_v2.0_flc.fits') as hdu:
    v2_uvis1 = hdu[4].data
    v2_uvis2 = hdu[1].data
with fits.open('idv404axq_flc.fits') as hdu:
    v1_uvis1 = hdu[4].data
    v1_uvis2 = hdu[1].data 
with fits.open('idv404axq_flt.fits') as hdu:
    flt_uvis1 = hdu[4].data
    flt_uvis2 = hdu[1].data 
    
# Load pixel area maps 
PAM_uvis1 = fits.getdata('example/UVIS1wfc3_map.fits')
PAM_uvis2 = fits.getdata('example/UVIS2wfc3_map.fits')

# Stich UVIS1 and 2 together and multiply by pixel area map
v2sci = np.concatenate([v2_uvis2*PAM_uvis2 ,v2_uvis1*PAM_uvis1])
v1sci = np.concatenate([v1_uvis2*PAM_uvis2 ,v1_uvis1*PAM_uvis1])
fltsci = np.concatenate([flt_uvis2*PAM_uvis2,flt_uvis1*PAM_uvis1])


<a id="bkgsub"></a>
## 11.3 Display 50x50 Pixel Background Subsection
One of the differences between the v1.0 and v2.0 CTE correction is the background level. Here, we show a square 50x50 pixel <br> subsection that is mostly featureless, i.e. no known sources or cosmic ray hits. The v2.0 correction significantly reduces noise <br> amplification and improves the resulting background. To aid in the visual inspection of the background subsection, an animated <br> GIF is included in the notebook that blinks between the v1.0 and v2.0 FLC files. 

In [ ]:
# Generate subplots
fig, [ax1,ax2,ax3] = plt.subplots(1,3,figsize=(15,10),dpi=150)

# Generate background subsections
flt_bkg = fltsci[2070:2120,2180:2230]
v1_bkg = v1sci[2070:2120,2180:2230]
v2_bkg = v2sci[2070:2120,2180:2230]

# Calculate min and max values for image scaling 
v1z1,v1z2 = zscale.zscale(v1_bkg)

# Display background subsection
im1 = ax1.imshow(flt_bkg,origin='lower',cmap='Greys_r',vmin=v1z1,vmax=v1z2)
im2 = ax2.imshow(v1_bkg,origin='lower',cmap='Greys_r',vmin=v1z1,vmax=v1z2)
im3 = ax3.imshow(v2_bkg,origin='lower',cmap='Greys_r',vmin=v1z1,vmax=v1z2)

# Formatting
fig.colorbar(im1,ax=ax1,shrink=0.35,pad=0.02)
fig.colorbar(im2,ax=ax2,shrink=0.35,pad=0.02)
fig.colorbar(im3,ax=ax3,shrink=0.35,pad=0.02)
ax1.set_title('FLT File BKG Subsection',size=14)
ax2.set_title('v1.0 PCTE FLC BKG Subsection',size=14)
ax3.set_title('v2.0 PCTE FLC BKG Subsection',size=14)
ax1.axis('off'),ax2.axis('off'),ax3.axis('off')
fig.tight_layout()


<div style="text-align: left"> <strong> <font size ="4">Animated GIF of the v1.0 and v2.0 FLC image subsections: </font> </strong> </div>

<img align="left" width="800" height="800" src="example/v1_v2_bkg.gif">



<a id="bkgdist"></a>
## 11.3.1 Pixel Distribution of Background Subsections
To gain a more quantitative picture of how the background pixels are changing between the different file versions, we plot <br>the distribution of pixel values from the 50x50 pixel subsection above. The increased noise level in the v1.0 correction is <br> apparent in the blue histogram below. In addition, we have also computed the difference between the file types and have <br> plotted them as histograms. These two plots illustrates how the background in the v2.0 CTE corrected image is less noisy <br> and more in-line with the values seen in the FLT image.    

In [ ]:
# Generate subplots
fig, [ax1,ax2] = plt.subplots(2,1,figsize=(7,10),dpi=120)

# Plot background subsection histograms
ax1.hist(flt_bkg.ravel(),bins=100,range=(-30,100),histtype='step',color='C3',label='FLT')
ax1.hist(v1_bkg.ravel(),bins=100,range=(-30,100),histtype='step',color='C0',label='v1.0 FLC')
ax1.hist(v2_bkg.ravel(),bins=100,range=(-30,100),histtype='step',color='k',label='v2.0 FLC')

# Plot background subsection differential histograms
ax2.hist((v1_bkg-flt_bkg).ravel(),bins=100,range=(-25,50),histtype='step',color='magenta',label='v1.0 FLC $-$ FLT')
ax2.hist((v2_bkg-flt_bkg).ravel(),bins=100,range=(-25,50),histtype='step',color='limegreen',label='v2.0 FLC $-$ FLT')
ax2.hist((v1_bkg-v2_bkg).ravel(),bins=100,range=(-25,50),histtype='step',color='C9',label='v1.0 $-$ v2.0 FLC')

# Formatting
ax1.set_title('Background Subsection Histogram',size=14)
ax2.set_title('Background Subsection Differential Histogram',size=14)
ax1.set_xlabel('Pixel Value [e-]',size=12)
ax1.set_ylabel('Frequency',size=12)
ax2.set_xlabel('Pixel Value [e-]',size=12)
ax2.set_ylabel('Frequency',size=12)
ax1.grid(alpha=0.5),ax2.grid(alpha=0.5)
ax1.legend(), ax2.legend()
ax1.set_yscale('log')
ax2.set_yscale('log')

fig.tight_layout()


<a id="imgsub"></a>
## 11.4 Display Image Subsection
Another difference between the v1.0 and v2.0 pixel-based CTE model is the amount of correction applied to fainter sources. <br> In order to reduce noise amplification, the fluxes of faint sources only receive a limited amount of correction in the v2.0 version. <br> We again provide an animated GIF of the two subsections to aid with the visual comparison. 

In [ ]:
# Generate subplots
fig, [ax1,ax2] = plt.subplots(2,1,figsize=(8,8),dpi=150)

# Calculate min and max values for image scaling 
v1z1,v1z2 = zscale.zscale(v1_uvis1)

# Display subsection
im1 = ax1.imshow(v1_uvis1,origin='lower',cmap='Greys_r',vmin=v1z1,vmax=v1z2)
im2 = ax2.imshow(v2_uvis1,origin='lower',cmap='Greys_r',vmin=v1z1,vmax=v1z2)

# Formatting
ax1.set_xlim(85,325),ax2.set_xlim(85,325)
ax1.set_ylim(0,149),ax2.set_ylim(0,149)
fig.colorbar(im1,ax=ax1,shrink=0.95,pad=0.01)
fig.colorbar(im2,ax=ax2,shrink=0.95,pad=0.01)
ax1.set_title('v1.0 PCTE FLC UVIS1 Subsection',size=14)
ax2.set_title('v2.0 PCTE FLC UVIS1 Subsection',size=14)
fig.tight_layout()


<div style="text-align: left"> <strong> <font size ="4">Animated GIF of the v1.0 and v2.0 FLC image subsections: </font> </strong> </div>
<img align="left" width="975" height="975" src="example/v1_v2_subsection.gif">

<a id="apphot"></a>
## 11.5 Aperture Photometry 
To show the quantitative difference in the observed flux of sources between the v1.0 and v2.0 pixel-based CTE correction,<br> we perform aperture photometry on six stars of varying brightness within the image subsection above. Here, we display the <br> image subsection again with the apertures, annuli, and star labels overplotted. This subsection includes the first ~150 rows <br> of UVIS 1, which means all of these stars suffer the most CTE flux loss as they transfer 1900+ rows to the readout amplifier.   

<img align="left" width="900" height="900" src="example/apphot_image.png">

First, we approximate the center x and y positions of the stars in the 4Kx4K science arrays, and create the apertures and <br> annuli using `photutils`. <br>

Then, we use `photutils` to measure the signal in each area and subtract the background values in the annuli from the <br>aperture sum values. We define a function named `get_flux()` to do this for us.

In [ ]:
def get_flux(data,aperture,annulus_aperture):
    """
    Function to calculate background subtracted aperture sum 
    
    Parameters:
    -----------
    data : float array
        The 2d array of science pixels being measured
    aperture : photutils obj
        A photutils aperture object with defined position and radius
    annulus_aperture : photutils obj
        A photutils circular annulus aperture object with defined position and radii
        
    Return:
    -------
    flux : float
        The measured background subtracted aperture sum 
    """
    
    # Generate photutils.aperture_photometry table object
    phot = aperture_photometry(data, aperture)
    
    # Measure background around sources. aperture_stats_tbl() comes from background_median.py
    bkg_phot = aperture_stats_tbl(data, annulus_aperture, method = 'exact', sigma_clip = True)
    
    # Calculate background subtracted aperture sum
    flux = phot['aperture_sum'] - bkg_phot['aperture_median'] * aperture.area
    
    return flux

# Approximate x,y pixel locations of each star in the 4Kx4K array
positions = [(299.4,2135.6),
             (114.7,2093.4),
             (171.3,2074.9),
             (262.6,2164.9),
             (289.1,2085.6),
             (204.8,2073.1)]

# Photutils cirular aperture object with small radius
aperture = CircularAperture(positions, r=3)

# Photutils circular annulus aperture object 
annulus_aperture = CircularAnnulus(positions, r_in=13, r_out=19)

# Call function to calculate flux of stars
# THE RUNTIME WARNING MAY BE IGNORED
fltflux = get_flux(fltsci,aperture,annulus_aperture)
v1flux = get_flux(v1sci,aperture,annulus_aperture)
v2flux = get_flux(v2sci,aperture,annulus_aperture)


<div class="alert alert-block alert-warning" style="color:black" > <b> You can ignore the <code>RuntimeWarning: invalid value encountered in true_divide</code> message.</b> <br> </div>
    

Finally, we plot our results.<br>

The first plot below shows the measured flux of the six stars in each of the different calibrated products. Stars 1-6 are organized <br> by increasing flux, with Star 1 being the faintest and Star 6 being the brightest. The x-axis values correspond to the median flux <br> value measured in each of the three different files (FLT, v1.0, and v2.0) per star. The y-axis values show the individual flux values <br> for each star where the different colors and symbols represent the three file types. For example, the individual flux values (y-axis) <br> of Star 1 for the FLT, v1.0, and v2.0 files are ~2467, 3056, and 2799 e- respectively. The median of the three values is ~2799 e-, <br> which is the value plotted on the x-axis. The second plot illustrates the percent differences between the three different <br> flux values per star, and uses the same x-axis values as the first plot. <br> 

The FLT files have no CTE correction applied and thus have the lowest measured flux for each star due to the CTE flux loss. <br> Stars 1-4 have noticably different flux values between the v1.0 and v2.0 CTE corrections. In v1.0, the CTE algorithm is actually <br> over-correcting these fainter sources resulting in higher aperture photometry measurements. The flux values for stars 1-4 range <br> from  ~3000 - 10000 e- within a 3-pixel radius aperture. The flux values between the v1.0 and v2.0 CTE corrections for stars <br> 1-4 have percent differences of ~9 - 4%. Sources with more than ~10000 e- within a 3-pixel aperture (stars 5 & 6) have more <br> compareable flux values between the v1.0 and v2.0 CTE corrections, and have percent differences less than 2%.

In [ ]:
# Generate subplots
fig, [ax1,ax2] = plt.subplots(2,1,figsize=(8,13),dpi=120)
ax1.grid(alpha=0.5,which='both'),ax2.grid(alpha=0.5)

# Find median flux values between products
medflux = np.median([fltflux,v1flux,v2flux],axis=0)

# Scatter plot of measured flux
ax1.scatter(medflux,fltflux,25,marker='o',c='C3',label='FLT')
ax1.scatter(medflux,v1flux,30,marker='^',c='C0',label='v1.0 FLC')
ax1.scatter(medflux,v2flux,45,marker='*',c='k',label='v2.0 FLC')

# Scatter plot of percentage difference
ax2.scatter(medflux,abs((fltflux-v1flux))/((fltflux+v1flux)/2)*100,30,
            marker='^',c='magenta',label=r'$\frac{|FLT - v1.0|}{(FLT + v1.0) ÷ 2}$')
ax2.scatter(medflux,abs((fltflux-v2flux))/((fltflux+v2flux)/2)*100,45,
            marker='*',c='limegreen',label=r'$\frac{|FLT - v2.0|}{(FLT + v2.0) ÷ 2}$')
ax2.scatter(medflux,abs((v1flux-v2flux))/((v1flux+v2flux)/2)*100,25,
            marker='s',c='C9',label=r'$\frac{|v1.0 - v2.0|}{(v1.0 + v2.0) ÷ 2}$')

# Formatting 
ax1.set_title('Measured Flux Within 3-pix Radius Aperture',size=14)
ax1.set_xlabel('Median Flux [e-]',size=12)
ax1.set_ylabel('Flux [e-]',size=12)
ax2.set_xlabel('Median Flux [e-]',size=12)
ax2.set_ylabel('Percent Difference [%]',size=12)
ax1.legend(prop={'size':11}),ax2.legend(prop={'size':15})
ax1.set_yscale('log')


<a id="conclude"></a>
## 12. Conclusions

Thank you for walking through this notebook. You now have everything you need to process your own files with the v1.0 CTE <br>correction within `calwf3`. Since completing this notebook you should be more familiar with:

- Applying the v1.0 pixel-based CTE correction to 2009-2021 UVIS data.
- Which header keywords of the `raw` fits file to edit in order to use the v1.0 correction.
- How to find and download the appropriate `PCTETAB` and `DRKCFILE` reference files.
- Verifying your version of `calwf3` and calibrating a `raw` file with the v1.0 CTE correction.
- Investigating the differences between v1.0 and v2.0 CTE corrected files.

### Congratulations, you have completed the notebook!

<a id="resources"></a>
## Additional Resources
Below are some additional resources that may be helpful. Please feel free to contact the [WFC3 Helpdesk](https://stsci.service-now.com/hst) for any questions.

- [WFC3 CTE Website](https://www.stsci.edu/hst/instrumentation/wfc3/performance/cte)
- [HSTCAL GitHub](https://github.com/spacetelescope/hstcal)
- [Calwf3 Documentation](https://wfc3tools.readthedocs.io/en/latest/wfc3tools/calwf3.html)
- [WFC3 Website](https://www.stsci.edu/hst/instrumentation/wfc3)
- [WFC3 Instrument Handbook](https://hst-docs.stsci.edu/wfc3ihb)
- [WFC3 Data Handbook](https://hst-docs.stsci.edu/wfc3dhb)
- [STScI Jupyter-notebooks](https://github.com/spacetelescope/notebooks/tree/master/notebooks)
- [STScI Astroconda Channel](http://ssb.stsci.edu/astroconda)

<a id="about"></a>
## About this Notebook

**Author:** Benjamin Kuhn; WFC3 Instrument Team

**Updated on:** January 19, 2023

<a id="cite"></a>
## Citations

If you use `numpy`, `astropy`, `astroquery`, `matplotlib`, or photutils for published research, please cite the authors.<br>
Follow these links for more information about citing the libraries below:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `astroquery`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)
* [Citing `matplotlib`](https://matplotlib.org/stable/users/project/citing.html)
* [Citing `numpy`](https://numpy.org/citing-numpy/)
* [Citing `photutils`](https://photutils.readthedocs.io/en/stable/citation.html)
<br>
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 